In [ ]:
import torch
from torch import nn
from torch import optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [ ]:
# 下面是最简单的CNN模型，有4做卷积的层和1个做全连接的层。
# 做卷积的层都是：卷积层 + 激活层 + 池化层
# 做全连接的层用了3个全连接，把维度从 256-> 512 -> 64 -> 10
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.layer1 = nn.Sequential()                                         # size = 3 * 32 * 32
        self.layer1.add_module('conv1', nn.Conv2d(3, 32, 3, 1, padding=1))    # in_channel, out_channel, kernel_size, stride, padding
        self.layer1.add_module('relu1', nn.ReLU(True))
        self.layer1.add_module('pool1', nn.MaxPool2d(2,2))                    # size = 32 * 16 * 16
        
        self.layer2 = nn.Sequential()
        self.layer2.add_module('conv2', nn.Conv2d(32, 64, 3, 1, padding=1))
        self.layer2.add_module('relu2', nn.ReLU(True))
        self.layer2.add_module('pool2', nn.MaxPool2d(2,2))                    # size = 64 * 8 * 8
        
        self.layer3 = nn.Sequential()
        self.layer3.add_module('conv3', nn.Conv2d(64, 128, 3, 1, padding=1))
        self.layer3.add_module('pool3', nn.MaxPool2d(True))
        self.layer3.add_module('relu3', nn.ReLU(True))                        # size = 128 * 4 * 4
        
        self.layer4 = nn.Sequential()
        self.layer4.add_module('conv4', nn.Conv2d(128, 256, 3, 1, padding=1))
        self.layer4.add_module('pool4', nn.MaxPool2d(True))
        self.layer4.add_module('relu4', nn.ReLU(True))                        # size = 256 * 2 * 2
        
        self.layer5 = nn.Sequential()
        self.layer5.add_module('fc1', nn.Linear(256*2*2, 512))
        self.layer5.add_module('fc_relu1', nn.ReLU(True))
        self.layer5.add_module('fc2', nn.Linear(512, 64))
        self.layer5.add_module('fc_relu2', nn.ReLU(True))
        self.layer5.add_module('fc3', nn.Linear(64, 10))

    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight.data)
                nn.init.xavier_normal_(m.weight.data)
                nn.init.kaiming_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.fill_(0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, 0, 0.01)
                m.bias.data.zero_()
            else:
                pass
        
        
    def forward(self, x):
        conv1 = self.layer1(x)
        conv2 = self.layer2(conv1)
        conv3 = self.layer3(conv2)
        conv4 = self.layer4(conv3)
        fc_input = conv4.view(conv4.size(0), -1)                            # 把数据展开成一维的
        fc_out = self.layer5(fc_input)
        return fc_out

In [ ]:
# 实例化一个模型
model = SimpleCNN()

In [ ]:
# 以下可以列出模型的具体组成
print(model)
print(model.modules)
print(model.children)
print(model.named_children)
print(model.named_modules)

In [ ]:
# 以下可以列出模型的前两层
new_model = nn.Sequential(*list(model.children())[:2])
print(new_model)

In [ ]:
# 把模型中属于Conv2d的层列出来。不过新版本的层名不支持. 所以要换一下
conv_model = nn.Sequential()
for layer in model.named_modules():
    if isinstance(layer[1], nn.Conv2d):
        conv_model.add_module(layer[0].replace('.','-'), layer[1])
print(conv_model)

In [ ]:
# 把模型中所有参数名打印出来
for param in model.named_parameters():
    print(param[0])

In [ ]:
# 对权重做初始化
model.init_weights()

In [ ]:
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        print(m)
        print(m.weight.shape)
        print(m.bias.shape)

In [102]:
# 下面是最简单的CNN模型，有4做卷积的层和1个做全连接的层。
# 做卷积的层都是：卷积层 + 激活层 + 池化层
# 做全连接的层用了3个全连接，把维度从 256-> 512 -> 64 -> 10
class MNISTCNN(nn.Module):
    def __init__(self):
        super(MNISTCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )   
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(128*7*7, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 10)
        )

    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight.data)
                nn.init.xavier_normal_(m.weight.data)
                nn.init.kaiming_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.fill_(0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, 0, 0.01)
                m.bias.data.zero_()
            else:
                pass
        
        
    def forward(self, x):
        x = self.layer1(x)                                  # 16 * 28 * 28
        x = self.layer2(x)                                  # 32 * 14 * 14
        x = self.layer3(x)                                  # 64 * 14 * 14
        x = self.layer4(x)                                  # 128 * 7 * 7
        x = x.view(x.size(0), -1)                            # 把数据展开成一维的
        x = self.fc(x)
        return x

In [103]:
# 开始准备数据
batch_size = 64
lr = 1e-2
num_epoches = 10

data_tf = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5],[0.5])])

train_dataset = datasets.MNIST(root='./data', train=True, transform=data_tf, download=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=data_tf)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = MNISTCNN()
# model = Batch_Net(28*28, 300, 100, 10)

loss_func = nn.CrossEntropyLoss()

opti_func = optim.Adam(model.parameters(), lr=lr)


In [105]:
# 开始训练
train_loss = list(range(num_epoches))
for epoch in range(num_epoches):
    train_loss[epoch] = 0
    for train_data in train_loader:
        img, label = train_data
        # img = img.view(img.size(0), -1)
        img = Variable(img)
        label = Variable(label)
        ##### forward #####
        out = model(img)
        loss = loss_func(out, label)
        ##### backward ###
        opti_func.zero_grad()
        loss.backward()
        opti_func.step()
    print("train loss: {}".format(loss))
    train_loss[epoch] = loss.data
print(train_loss)
    
# Simple_Net      : train loss: 0.49769243597984314
# Activation_Net  : train loss: 0.025277957320213318
# Batch_Net.      : train loss: 0.05666544288396835

train loss: 0.05323347449302673
train loss: 0.024843575432896614
train loss: 0.10679031908512115
train loss: 0.004946841858327389
train loss: 7.729105709586293e-05
train loss: 0.019284864887595177
train loss: 0.059963155537843704
train loss: 0.00047266847104765475
train loss: 0.05882607400417328
train loss: 0.0003062101313844323
[tensor(0.0532), tensor(0.0248), tensor(0.1068), tensor(0.0049), tensor(7.7291e-05), tensor(0.0193), tensor(0.0600), tensor(0.0005), tensor(0.0588), tensor(0.0003)]


In [107]:
# 开始测试
model.eval()
eval_loss = 0
eval_acc = 0

for data in test_loader:
    img, label = data
    # img = img.view(img.size(0), -1)
    img = Variable(img)
    label = Variable(label)
    
    out = model(img)
    loss = loss_func(out, label)
    eval_loss += loss.data * label.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred==label).sum()
    eval_acc += num_correct.data

print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss/(len(test_dataset)), eval_acc/(len(test_dataset))))

# Simple_Net      : Test Loss: 0.505879, Acc: 0.849400
# Activation_Net  : Test Loss: 0.207220, Acc: 0.950500
# Batch_Net       : Test Loss: 0.087046, Acc: 0.977900

Test Loss: 0.060916, Acc: 0.988600
